# 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import os
import random
import contextlib
import types
from pathlib import Path

import torch
import submitit

from cupbearer import data, detectors, models, scripts, tasks, utils

/apps/Arch/software/tqdm/4.64.1-GCCcore-12.2.0/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
jobs = []

In [4]:
gpu_executor = submitit.SlurmExecutor(folder="runs")
gpu_executor.update_parameters(
    account="NAISS2023-22-1064",
    gpus_per_node="A40:1",#"A100:1",
    time=12*60,  # minutes
)

In [5]:
cpu_executor = submitit.SlurmExecutor(folder="runs")
cpu_executor.update_parameters(
    account="NAISS2023-22-1064",
    ntasks_per_node=1,
    cpus_per_task=16,
    constraint="NOGPU",
    time=2*24*60,  # minutes
)

In [6]:
def get_detectors(model: models.HookedModel) -> detectors.AnomalyDetector:
    default_kwargs = {
        "batch_size": 256,
        "num_workers": 12,
        "max_epochs": 50,
    }
    if not isinstance(model, models.PreActResNet):
        detector = detectors.AbstractionDetector(
            abstraction=detectors.abstraction.LocallyConsistentAbstraction.get_default(
                model,
                size_reduction=2,
            ),
        )
        train_kwargs = default_kwargs
        yield detector, train_kwargs
    
        detector = detectors.AbstractionDetector(
            abstraction=detectors.abstraction.AutoencoderAbstraction.get_default(
                model,
                size_reduction=2,
            ),
        )
        train_kwargs = default_kwargs
        yield detector, train_kwargs

    detector = detectors.FinetuningAnomalyDetector()
    def configure_optimizers(self):
        # Helper to monkey patch another optimizer
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.lr,
            momentum=0.9,
            weight_decay=5e-4,
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer=optimizer,
            T_max=100,
        )
        return {"optimizer": optimizer, "lr_scheduler": scheduler}

    train_kwargs = default_kwargs
    train_kwargs.update({
        "batch_size": 256,
        "lr": 0.01,
        "num_workers": 12,
        "max_epochs": 100,
        "configure_optimizers": configure_optimizers,
    })
    yield detector, train_kwargs

    
    if isinstance(model, models.PreActResNet):
        detector_kwargs = {"activation_name_func": lambda model: ["res4"]}
    elif isinstance(model, models.CNN):
        detector_kwargs = {"activation_name_func": lambda model: ["post_global_pool"]}
    else:
        detector_kwargs = {}
    
    detector = detectors.MahalanobisDetector(**detector_kwargs)
    train_kwargs = default_kwargs
    yield detector, train_kwargs
    
    detector = detectors.SpectralSignatureDetector(**detector_kwargs)
    train_kwargs = default_kwargs
    yield detector, train_kwargs
    
    detector = detectors.QuantumEntropyDetector(**detector_kwargs)
    train_kwargs = default_kwargs
    yield detector, train_kwargs

# 

In [7]:
def train_detector(model, Dataset, Backdoor, detector, train_kwargs, classifier_path):
    
    if torch.cuda.is_available():
        torch.set_float32_matmul_precision('high')
        
    # Load backdoored classifier
    models.load(model, classifier_path)

    if Backdoor == data.WanetBackdoor:
        backdoor = Backdoor(path=classifier_path, p_backdoor=1.0, p_noise=0.0)
        task_name = Backdoor.__name__
    elif Backdoor is not None:
        backdoor = Backdoor(p_backdoor=1.0)
        task_name = Backdoor.__name__
    else:
        task_name = "Adversarial"
    
    # Detector save path
    base_path_parts = list(classifier_path.parts)
    base_path_parts[3] = task_name
    detector_path = os.path.join(
        *base_path_parts,
        f"detector",
        f"{type(detector if not isinstance(detector, detectors.AbstractionDetector) else detector.abstraction).__name__}",
        os.getenv("SLURM_JOB_ID", datetime.datetime.now().isoformat()),
    )

    # Initialize task
    if Backdoor is not None:
        task = tasks.backdoor_detection(
            model=model,
            train_data=Dataset(train=True),
            test_data=Dataset(train=False),
            backdoor=backdoor,
            trusted_fraction=0.9,  # [0.9, 0.95, 0.99, 0.995, 0.999]
        )
    else:
        task = tasks.adversarial_examples(
            model=model,
            train_data=Dataset(train=True),
            test_data=Dataset(train=False),
            cache_path=classifier_path.parent,
            trusted_fraction=0.9,  # [0.9, 0.95, 0.99, 0.995, 0.999]
        )
    
    # Train detector
    scripts.train_detector(
        task=task,
        detector=detector,
        save_path=detector_path,
        eval_batch_size=1024,
        verbose=True,
        num_classes=Dataset.num_classes,
        **train_kwargs,
    )

In [8]:
#train_detector(random.choice(classifier_paths))

In [9]:
executor = gpu_executor
n_reps = 5
with executor.batch():
    for _ in range(n_reps):
        for Dataset in [
            data.MNIST,
            data.GTSRB,
            data.CIFAR10,
        ]:
            for model in [
                models.MLP(
                    input_shape=(3, 28, 28) if Dataset == data.MNIST else (3, 32, 32),
                    hidden_dims=[128, 128, 128],
                    output_dim=Dataset.num_classes,
                ),
                models.CNN(
                    input_shape=(3, 28, 28) if Dataset == data.MNIST else (3, 32, 32),
                    channels=[16, 32, 32, 64],
                    dense_dims=[128],
                    output_dim=Dataset.num_classes,),
                models.PreActResNet(
                    block=models.models.PreActBlock,
                    num_blocks=[2, 2, 2, 2],
                    num_classes=Dataset.num_classes,
                ),
            ]:
                for Backdoor in [
                    data.CornerPixelBackdoor,
                    data.NoiseBackdoor,
                    data.WanetBackdoor,
                ]:
                    for detector, train_kwargs in get_detectors(model):
                        if Backdoor == data.WanetBackdoor:
                            classifier_paths = Path(f"logs/{type(model).__name__}/{Dataset.__name__}/{Backdoor.__name__}/").glob("2[3-9]*/wanet_backdoor.pt")
                            try:
                                classifier_path = random.choice(list(classifier_paths)).parent
                            except IndexError:
                                pass
                        else:
                            classifier_paths = Path(f"logs/{type(model).__name__}/{Dataset.__name__}/{Backdoor.__name__}/").glob("2[3-9]*/checkpoints/last.ckpt")
                            try:
                                classifier_path = random.choice(list(classifier_paths)).parents[1]
                            except IndexError:
                                pass

                        if True:
                            job = executor.submit(
                                train_detector,
                                Dataset=Dataset,
                                model=model,
                                Backdoor=Backdoor,
                                detector=detector,
                                train_kwargs=train_kwargs,
                                classifier_path=classifier_path,
                            )
                            jobs.append(job)
                        else:
                            train_detector(
                                Dataset=Dataset,
                                model=model,
                                Backdoor=Backdoor,
                                detector=detector,
                                train_kwargs=train_kwargs,
                                classifier_path=classifier_path,
                            )

                for detector, train_kwargs in get_detectors(model):
                    classifier_paths = Path(f"logs/{type(model).__name__}/{Dataset.__name__}/WanetBackdoor/").glob("2[3-9]*/checkpoints/last.ckpt")
                    try:
                        classifier_path = random.choice(list(classifier_paths)).parents[1]
                    except IndexError:
                        pass
                        
                    if True:
                        job = executor.submit(
                            train_detector,
                            Dataset=Dataset,
                            model=model,
                            Backdoor=None,
                            detector=detector,
                            train_kwargs=train_kwargs,
                            classifier_path=classifier_path,
                        )
                        jobs.append(job)
                        


In [10]:
for job in jobs:
    print(job)

SlurmJob<job_id=2353690_0, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_1, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_2, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_3, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_4, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_5, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_6, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_7, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_8, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_9, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_10, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_11, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_12, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_13, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_14, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_15, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_16, task_id=0, state="UNKNOWN">
SlurmJob<job_id=2353690_17, task_id=0, state="UNKNOWN">
Sl

In [11]:
#for job in jobs:
#    if job.state == "PENDING":
#        job.cancel()

In [12]:
job = jobs[-1]

In [13]:
print('\n'.join(job.stdout().split('\n')[-100:]))

AttributeError: 'NoneType' object has no attribute 'split'

In [ ]:
print(job.stderr())

In [ ]:
for job in jobs:
    if job.state == "FAILED":
        print(job)
        print(job.stderr())

In [ ]:
!job_stats.py {' '.join([job.job_id for job in jobs])}

In [ ]:
! scontrol show job {job.job_id}

In [ ]:
cov = torch.load('covariance_res4_2114226.pt')
cov.size()

In [ ]:
!ls covariance*.pt
32*32*64

In [ ]:
cov = torch.load('covariance_res4_2114226.pt')
print(cov.size())
%timeit -n 1 -r 1 torch.linalg.pinv(cov.to('cpu'), hermitian=True).to('cpu')
del cov

In [ ]:
model = models.PreActResNet(models.models.PreActBlock, [2, 2, 2, 2], num_classes=Dataset.num_classes)
act = get_detector(model)[0].get_activations(torch.randn(2, 3, 32, 32))[1]['res4']

In [ ]:
mean = act.mean(0)
cov = (act - act.mean(0)).flatten(1).T @ (act - act.mean(0)).flatten(1)

In [ ]:
torch.linalg.pinv(cov.to('cuda'), rcond=1e-5, hermitian=True)